### 통합 코드_v2

In [2]:
import os
import joblib
import pickle
from dotenv import load_dotenv
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from openai import OpenAI

# ------------------------------------------
# 0. 환경 설정 및 OpenAI API 초기화
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = None
try:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI 클라이언트 초기화 완료")
except Exception as e:
    print("OpenAI 클라이언트 초기화 실패:", e)

# ------------------------------------------
# 1. MBTI 예측 모델 및 토크나이저 로딩
mbti_model = load_model("mbti_predictor.h5")
with open("mbti_tokenizer.pkl", "rb") as f:
    mbti_tokenizer = pickle.load(f)
MAX_LEN_MBTI = 100

idx2mbti = {
    0: "INFP", 1: "ENFP", 2: "INFJ", 3: "ENFJ",
    4: "INTP", 5: "ENTP", 6: "INTJ", 7: "ENTJ",
    8: "ISFP", 9: "ESFP", 10: "ISTP", 11: "ESTP",
    12: "ISFJ", 13: "ESFJ", 14: "ISTJ", 15: "ESTJ"
}

def preprocess_mbti(text, tokenizer):
    seq = tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=MAX_LEN_MBTI, padding='post')

def predict_mbti(text):
    x = preprocess_mbti(text, mbti_tokenizer)
    pred = mbti_model.predict(x)[0]
    idx = pred.argmax()
    return idx2mbti.get(idx, "INFP")

# ------------------------------------------
# 2. 감정 분석 모델 및 벡터라이저 로딩 (joblib)
emotion_model = joblib.load("emotion_model.pkl")
vectorizer_emotion = joblib.load("emotion_vectorizer.pkl")

def analyze_sentiment(text):
    x = vectorizer_emotion.transform([text])
    pred = emotion_model.predict(x)
    return pred[0]

# ------------------------------------------
# 3. 상담 모델 및 토크나이저 로딩 (keras)
counsel_model = load_model("counsel_model_keras.h5")
with open("keras_tokenizer.pkl", "rb") as f:
    counsel_tokenizer = pickle.load(f)
MAX_LEN_COUNSEL = 50

def preprocess_counsel(text):
    seq = counsel_tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=MAX_LEN_COUNSEL, padding='post')

def generate_counsel_response(user_input):
    x = preprocess_counsel(user_input)
    pred = counsel_model.predict(x)[0]
    # pred 처리 방법에 따라 다름, 예: 인덱스 argmax or 특정 decoding 필요
    # 여기서는 예시로 argmax 후 간단한 응답 매핑 가정
    response_idx = pred.argmax()
    # 간단한 매핑 예시 (실제 프로젝트에 맞게 수정 필요)
    response_map = {
        0: "당신의 고민을 잘 이해했습니다. 조금만 더 힘내세요!",
        1: "지금 상황이 어렵겠지만 분명히 좋은 결과가 있을 거예요.",
        2: "긍정적인 마음으로 상황을 바라보는 것이 중요합니다.",
    }
    return response_map.get(response_idx, "제가 이해하기 어려운 내용입니다. 조금 더 구체적으로 말씀해 주세요.")

# ------------------------------------------
# 4. GPT 상담 응답 생성기 (필요 시 사용)
MBTI_TONE = {
    "ENFP": "따뜻하고 유쾌하며 이모티콘을 자주 사용합니다.",
    "ISTJ": "분석적이고 신중하며 단정한 말투입니다.",
    "INFP": "섬세하고 감정에 공감하는 부드러운 말투입니다.",
    "ESTJ": "단호하고 체계적이며 사실 위주의 말투입니다.",
    "INTP": "논리적이고 중립적인 말투입니다.",
    "ESFJ": "친근하고 배려심 많은 말투로 위로를 잘 전합니다.",
    "ENTP": "재치 있고 유머러스하며 아이디어를 자유롭게 표현합니다.",
    "ISFJ": "조용하지만 따뜻하고 배려 깊은 말투로 상대를 존중합니다.",
    "INFJ": "직관적이며 깊이 있는 표현과 따뜻한 공감이 어우러진 말투입니다.",
    "ESTP": "직설적이고 에너지 넘치며 상황 중심적으로 조언합니다.",
    "ISFP": "차분하고 부드러우며 감정에 민감하게 반응합니다.",
    "INTJ": "간결하고 직관적인 말투이며 효율 중심적으로 접근합니다.",
    "ENTJ": "자신감 있고 목표 지향적이며 명확한 표현을 사용합니다.",
    "ENFJ": "따뜻하고 포용적인 말투로 감정에 깊이 공감합니다.",
    "ISTP": "과묵하고 실용적인 조언 위주로 핵심만 전달합니다.",
    "ESFP": "밝고 생동감 있는 말투로 친근하고 즉흥적인 표현을 자주 사용합니다."
}

MBTI_OPPOSITE_MAP = {
    "ISTJ": "ENFP", "ISFJ": "ENTP", "INFJ": "ESTP", "INTJ": "ESFP",
    "ISTP": "ENFJ", "ISFP": "ENTJ", "INFP": "ESTJ", "INTP": "ESFJ",
    "ESTP": "INFJ", "ESFP": "INTJ", "ENFP": "ISTJ", "ENTP": "ISFJ",
    "ESTJ": "INFP", "ESFJ": "INTP", "ENFJ": "ISTP", "ENTJ": "ISFP"
}

def get_tf_trait(mbti): 
    return "F" if mbti[2] == "F" else "T"

def generate_gpt_response(user_input, mbti):
    if client is None:
        return "OpenAI API 키가 설정되지 않았습니다."

    tf_trait = get_tf_trait(mbti)
    tone = MBTI_TONE.get(MBTI_OPPOSITE_MAP.get(mbti, "ENFP"))
    instruction = "감정에 공감하며 부드럽게 위로해주세요." if tf_trait == "F" else "논리적으로 조언해주세요."
    system_msg = f"{MBTI_OPPOSITE_MAP.get(mbti)} 말투의 심리상담 챗봇입니다.\n말투: {tone}\n{instruction}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_input}
        ],
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

# ------------------------------------------
# 5. 사용자 입력 예시 & 실행

if __name__ == "__main__":
    print("\n안녕하세요! MBTI 예측 챗봇입니다.\n아래에 5개 문장을 입력해 주세요:")
    session_inputs = [input(f"문장 {i+1}: ").strip() for i in range(5)]
    combined_text = " ".join(session_inputs)
    predicted_mbti = predict_mbti(combined_text)
    print(f"\n[예측된 MBTI]: {predicted_mbti}")

    user_input = input("\n고민을 입력하세요: ")

    detected_emotion = analyze_sentiment(user_input)
    print(f"\n[감정 분석 결과]: {detected_emotion}")

    # 상담 답변: 두 가지 방식 중 선택
    # 1) keras 상담 모델 기반
    counsel_reply = generate_counsel_response(user_input)
    print("\n[상담 모델 응답]:")
    print(counsel_reply)

    # 2) OpenAI GPT API 상담 응답 (원하면 주석 해제해서 사용)
    # gpt_reply = generate_gpt_response(user_input, predicted_mbti)
    # print("\n[GPT 상담 응답]:")
    # print(gpt_reply)

OpenAI 클라이언트 초기화 완료


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'mbti_predictor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)